In [ ]:
from flask import Flask, request, render_template
from PIL import Image
import numpy as np
import tensorflow as tf
import io
import os
import base64
x= ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']
app = Flask(__name__)
model = tf.keras.models.load_model('braintumor')
@app.route('/',methods=['GET'])
def home():
    return render_template('home.html')
@app.route('/<path:filename>')
def serve_static(filename):
    root_dir = os.path.dirname(os.getcwd())
    return send_from_directory(os.path.join(root_dir, 'templates'), filename)
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    result_display = False 
    predict = ""  
    img_data = None 
    a=False
    b=False
    c=False
    if request.method == 'POST':
        file = request.files['image']
        if file:
            img = Image.open(file.stream).convert('RGB')
            img = img.resize((150, 150))
            img_array = np.array(img)
            img_array = np.expand_dims(img_array, axis=0)

            prediction = model.predict(img_array)
            class_index = np.argmax(prediction)
            result_display = True
            if class_index==0:
                a=True
            if class_index==2:
                b=True
            if class_index==3:
                c=True
            if class_index == 1:
                predict='The Person has no brain tumor'
            else:
                predict = 'The person has brain tumor'
            img_byte_array = io.BytesIO()
            img.save(img_byte_array, format='PNG')
            img_data = base64.b64encode(img_byte_array.getvalue()).decode('utf-8')

    return render_template('index.html', result_display=result_display, predict=predict, img_data=img_data,a=a,b=b,c=c) 

if __name__ == "__main__":
    app.run()
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
